# **Laboratorio 2**

Derek Arreaga - 22537

Paula Barillas - 22764

Mónica Salavatierra - 22249

##### LINK DE REPOSITORIO
https://github.com/FabianKel/LAB2-IA

# **TASK #1 - Preguntas Teóricas**

### ¿Por qué el modelo de Naive Bayes se le considera “naive”?

- El modelo se considera naive debido a que asume que las características de medición son independientes entre sí y contribuyen por igual en el momento de dar resultado. Esta suposición de independencia es rara vez es cierta. Sin embargo, permite que el modelo sea muy eficiente en términos de tiempo de computación y memoria.

### Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda ¿cómo funciona el truco del Kernel para este modelo? (Lo que se espera de esta pregunta es que puedan explicar en sus propias palabras la fórmula a la que llegamos que debemos optimizar de SVM en clase)

- La formulación matemática que se busca optimizar en Support Vector Machine (SVM) es maximizar el margen entre las clases o bien como fue mencionado en clase crear una frontera para separar las diferentes clases con las que estamos logrando. Esto se logra resolviendo el problema de optimización cuadrática donde se aseguran que los datos estén correctamente clasificados y fuera del margen. La función es:

$$\min_{(\mathbf{w}, b)} \frac{1}{2} \|\mathbf{w}\|^2$$

sujeto a:

$$y_i (\mathbf{w} \cdot \mathbf{x}_i + b) \geq 1 \quad \forall i$$

donde $\mathbf{w}$ es el vector de pesos, $b$ es el sesgo, $\mathbf{x}_i$ son los vectores de características y $y_i$ son las etiquetas de clase.

- El truco del Kernel permite a SVM manejar datos no linealmente separables al transformar los datos originales a un espacio de mayor dimensión donde es más probable que sean linealmente separables. Esto se hace utilizando una función de kernel $(K(\mathbf{x}_i, \mathbf{x}_j))$ que va calcula el producto punto en el espacio transformado sin necesidad de calcular explícitamente la transformación.
### Investigue sobre Random Forest y responda
1. ¿Qué tipo de ensemble learning es este modelo?
    - El tipo de ensemble learning es Bagging (Bootstrap Aggregating).

2. ¿Cuál es la idea general detrás de Random Forest?
    - La idea general detrás de Random Forest es construir múltiples árboles de decisión durante el entrenamiento y clasificación o la media de las predicciones (regresión) de los árboles individuales. Cada árbol es entrenado con una muestra aleatoria del dataset original con reemplazo (bootstrap sample), y en cada nodo de los árboles, solo un subconjunto aleatorio de características es considerado para la división.

3. ¿Por qué se busca baja correlación entre los árboles de Random Forest?
    - Se busca baja correlación entre los árboles de Random Forest porque si los árboles son altamente correlacionados, sus errores también serán correlacionados, lo que reduce la efectividad del ensemble. La baja correlación asegura que los errores de los árboles individuales se cancelen entre sí, mejorando la precisión y robustez del modelo final.



# **TASK #2 - Naive Bayes: Clasificador de Mensajes Ham/Spam**

Importación de librerias

In [1]:
import numpy as np
import math
import re
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

Importación de archivo a utilizar 

In [2]:
def load_dataset(filename):
    messages = []
    labels = []
    
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            label, message = line.strip().split('\t', 1)
            messages.append(message)
            labels.append(label)
    
    df = pd.DataFrame({"label": labels, "message": messages})
    
    return df  

# Cargar el dataset en un df
df = load_dataset('entrenamiento.txt')

print(df.head())

# Verificar la distribución de clases
print("\nDistribución de clases:")
print(df["label"].value_counts())

  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

Distribución de clases:
label
ham     4818
spam     747
Name: count, dtype: int64


## **Task 2.1 Limpieza y lectura de datos**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5565 non-null   object
 1   message  5565 non-null   object
dtypes: object(2)
memory usage: 87.1+ KB


In [4]:
df.describe()

,label,message
count,5565,5565
unique,2,5153
top,ham,"Sorry, I'll call later"
freq,4818,30


Preprocesar Texto

In [5]:
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    return text.split()

df["message"] = df["message"].apply(preprocess_text)

# Convertir listas de palabras en cadenas separadas por espacios
df["message"] = df["message"].apply(lambda words: " ".join(words))

print(df.head(10))

  label                                            message
0   ham  go until jurong point crazy available only in ...
1   ham                            ok lar joking wif u oni
2  spam  free entry in a wkly comp to win fa cup final ...
3   ham        u dun say so early hor u c already then say
4   ham  nah i dont think he goes to usf he lives aroun...
5  spam  freemsg hey there darling its been weeks now a...
6   ham  even my brother is not like to speak with me t...
7   ham  as per your request melle melle oru minnaminun...
8  spam  winner as a valued network customer you have b...
9  spam  had your mobile months or more u r entitled to...


Dataset dividido en training y test

In [6]:
# Training y Test
X_train, X_test, y_train, y_test = train_test_split(
    df["message"], df["label"], test_size=0.2, random_state=42, stratify=df["label"]
)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")
print(type(X_train))

Tamaño del conjunto de entrenamiento: 4452
Tamaño del conjunto de prueba: 1113
<class 'pandas.core.series.Series'>


## **Task 2.2 - Construcción del Modelo**

In [70]:

def TrainNaive_bayes(messages, labels, alpha=0.5):

    #contadores
    word_counts = {"ham": {}, "spam": {}}  # Diccionario para contar palabras por clase
    class_counts = {"ham": 0, "spam": 0}  # Contador de mensajes por clase
    vocabulary = set()  # Conjunto de palabras únicas

    # Recorrer los mensajes y contar palabras por clase
    for i in range(len(messages)):  
        message = messages[i]
        label = labels[i]

        # Contar cuántos mensajes hay en cada clase
        class_counts[label] += 1

        words = preprocess_text(message)

        # Contar palabras en cada categoría
        for word in words:
            if word not in word_counts[label]:
                word_counts[label][word] = 0
            word_counts[label][word] += 1
            vocabulary.add(word)  

    # Calcular probabilidades de cada clase
    total_messages = len(messages)
    class_probs = {label: class_counts[label] / total_messages for label in class_counts}

    # Calcular probabilidades de cada palabra 
    vocab_size = len(vocabulary) 
    word_probs = {"ham": {}, "spam": {}}

    for label in class_counts.keys():
        total_words = sum(word_counts[label].values())  
        
        for word in vocabulary:
            # Aplicar Laplace Smoothing
            word_count = word_counts[label].get(word, 0)  
            word_probs[label][word] = (word_count + alpha) / (total_words + alpha * vocab_size)

    return word_probs, class_probs, list(vocabulary)

# Convertir X_train y y_train en listas 
X_train = list(X_train) if isinstance(X_train, pd.Series) else X_train
y_train = list(y_train) if isinstance(y_train, pd.Series) else y_train

# Entrenar el modelo con el conjunto de entrenamiento
word_probs, class_probs, vocabulary = TrainNaive_bayes(X_train, y_train)

# estadísticas del modelo entrenado
print("Tamaño del vocabulario:", len(vocabulary))

Tamaño del vocabulario: 7533


### **Justificación de la métrica de desempeño a utilizar**
Tomando en cuenta que el dataset se encuentra desbalanceado (existe una mayor cantidad de mensajes ham que spam), se utilizará la métrica de **recall** para medir el desempeño del modelo:
- Al obtener métricas sólidas y favorables y entender que tan bien se clasifican los mensajes, podremos aplicar pesos para mejorar el rendimiento del modelo en la detección de la clase minoritaria (spam).
- El recall es clave en este caso, ya que nos interesa minimizar los falsos negativos en la detección de mensajes spam, evitando que estos sean clasificados incorrectamente como ham.


In [ ]:
manual_class_prior = {
    "ham": 0.3,  
    "spam": 0.7
}

def evaluate_naive_bayes(X_test, y_test, word_probs, class_probs, vocabulary, class_prior):
    # contadores para la matriz de confusión
    true_pos_spam = 0
    false_neg_spam = 0
    true_pos_ham = 0
    false_neg_ham = 0
    
    # Evaluar cada mensaje
    for message, true_label in zip(X_test, y_test):
        words = preprocess_text(message)
        
        scores = {
            "ham": math.log(class_prior["ham"]),  
            "spam": math.log(class_prior["spam"])
        }
        
        # Calcular probabilidad para cada clase usando Laplace Smoothing
        for label in ["ham", "spam"]:
            for word in words:
                if word in vocabulary:
                    scores[label] += math.log(word_probs[label][word])
        
        # Determinar predicción 
        pred_label = max(scores, key=scores.get)
        
        # Actualizar contadores
        if true_label == "spam":
            if pred_label == "spam":
                true_pos_spam += 1
            else:
                false_neg_spam += 1
        else:  # true_label == "ham"
            if pred_label == "ham":
                true_pos_ham += 1
            else:
                false_neg_ham += 1
    
    # Calcular métricas
    recall_spam = true_pos_spam / (true_pos_spam + false_neg_spam) if (true_pos_spam + false_neg_spam) > 0 else 0
    recall_ham = true_pos_ham / (true_pos_ham + false_neg_ham) if (true_pos_ham + false_neg_ham) > 0 else 0
    avg_recall = (recall_spam + recall_ham) / 2
    
    metrics = {
        "recall_spam": recall_spam,
        "recall_ham": recall_ham,
        "avg_recall": avg_recall,
    }
    
    return metrics

print("\n=== Entrenamiento manual  ===")
print("\n=== Evaluación en conjunto de entrenamiento ===")
train_metrics = evaluate_naive_bayes(X_train, y_train, word_probs, class_probs, vocabulary, manual_class_prior)
print(f"\nMétricas de evaluación:")
print(f"- Recall para spam: {train_metrics['recall_spam']:.3f}")
print(f"- Recall para ham: {train_metrics['recall_ham']:.3f}")
print(f"- Recall promedio: {train_metrics['avg_recall']:.3f}")

print("\n=== Evaluación en conjunto de prueba ===")
test_metrics = evaluate_naive_bayes(X_test, y_test, word_probs, class_probs, vocabulary, manual_class_prior)
print(f"\nMétricas de evaluación:")
print(f"- Recall para spam: {test_metrics['recall_spam']:.3f}")
print(f"- Recall para ham: {test_metrics['recall_ham']:.3f}")
print(f"- Recall promedio: {test_metrics['avg_recall']:.3f}")



=== Entrenamiento manual  ===

=== Evaluación en conjunto de entrenamiento ===

Métricas de evaluación:
- Recall para spam: 0.980
- Recall para ham: 0.969
- Recall promedio: 0.974

=== Evaluación en conjunto de prueba ===

Métricas de evaluación:
- Recall para spam: 0.933
- Recall para ham: 0.967
- Recall promedio: 0.950


## **Task 2.3 - Clasificación de mensajes futuros**

In [77]:
def classify_message(message, word_probs, class_probs, vocabulary):

    words = preprocess_text(message)
    
    # Calcular score para cada clase
    scores = {"ham": math.log(class_probs["ham"]), 
            "spam": math.log(class_probs["spam"])}
    
    # Para cada palabra en el mensaje
    for word in words:
        if word in vocabulary:
            for label in ["ham", "spam"]:
                # Multiplicar las probabilidades (se puede realizar una suma de logaritmos)
                scores[label] += math.log(word_probs[label][word])
    
    # Convertir scores logarítmicos a probabilidades
    total = sum(math.exp(score) for score in scores.values())
    probs = {label: math.exp(score)/total for label, score in scores.items()}
    
    # Determinar clasificación
    classification = max(probs.items(), key=lambda x: x[1])[0]
    
    return classification, probs["spam"], probs["ham"]


def classify_interface():
    while True:
        print("\n=== Clasificador de Mensajes ===")
        message = input("\nIngrese el mensaje a clasificar (o 'q' para salir): ")
        
        if message.lower() == 'q':
            break
            
        # Clasificar el mensaje
        classification, spam_prob, ham_prob = classify_message(
            message, word_probs, class_probs, vocabulary
        )
        
        print("\nResultados de la clasificación:")
        print(f"Mensaje ingresado: \"{message}\"")
        print(f"Probabilidad de spam: {spam_prob:.2%}")
        print(f"Probabilidad de ham: {ham_prob:.2%}")
        print(f"Clasificación final: {classification.upper()}")

classify_interface()


=== Clasificador de Mensajes ===

Resultados de la clasificación:
Mensaje ingresado: "Are you ready for tonight's party?"
Probabilidad de spam: 0.30%
Probabilidad de ham: 99.70%
Clasificación final: HAM

=== Clasificador de Mensajes ===

Resultados de la clasificación:
Mensaje ingresado: "Click here to claim your prize!"
Probabilidad de spam: 100.00%
Probabilidad de ham: 0.00%
Clasificación final: SPAM

=== Clasificador de Mensajes ===

Resultados de la clasificación:
Mensaje ingresado: "Congratulations! You won a new car!"
Probabilidad de spam: 99.56%
Probabilidad de ham: 0.44%
Clasificación final: SPAM

=== Clasificador de Mensajes ===


## **Task 2.4 - Entrenamiento con librerías**

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score

# Vectorizar el texto (convertir a matriz de características)
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

sklearn_model = MultinomialNB(alpha=0.5, class_prior=[0.3, 0.7])
sklearn_model.fit(X_train_vectorized, y_train)

y_train_pred = sklearn_model.predict(X_train_vectorized)
y_test_pred = sklearn_model.predict(X_test_vectorized)

# Calcular recall para cada conjunto
sklearn_train_recall_spam = recall_score(y_train, y_train_pred, pos_label='spam')
sklearn_train_recall_ham = recall_score(y_train, y_train_pred, pos_label='ham')
sklearn_train_recall_avg = recall_score(y_train, y_train_pred, average='macro')

sklearn_test_recall_spam = recall_score(y_test, y_test_pred, pos_label='spam')
sklearn_test_recall_ham = recall_score(y_test, y_test_pred, pos_label='ham')
sklearn_test_recall_avg = recall_score(y_test, y_test_pred, average='macro')


print("\n=== Entrenamiento con librerías ===")
print("\n=== Evaluación en conjunto de entrenamiento ===")
print(f"- Recall spam: {sklearn_train_recall_spam:.3f}")
print(f"- Recall ham: {sklearn_train_recall_ham:.3f}")
print(f"- Recall promedio: {sklearn_train_recall_avg:.3f}")
print("\n=== Evaluación en conjunto de prueba ===")
print(f"- Recall spam: {sklearn_test_recall_spam:.3f}")
print(f"- Recall ham: {sklearn_test_recall_ham:.3f}")
print(f"- Recall promedio: {sklearn_test_recall_avg:.3f}")


=== Entrenamiento con librerías ===

=== Evaluación en conjunto de entrenamiento ===
- Recall spam: 0.987
- Recall ham: 0.969
- Recall promedio: 0.978

=== Evaluación en conjunto de prueba ===
- Recall spam: 0.933
- Recall ham: 0.966
- Recall promedio: 0.949


### **¿Cuál implementación lo hizo mejor? ¿Su implementación o la de la librería?**
- Ambas implementaciones lograron métricas de recall muy similares en el conjunto de prueba, con diferencias mínimas.

    - La implementación manual obtuvo un recall promedio de 0.950 en el conjunto de prueba.
    - La implementación con librerías obtuvo un recall promedio de 0.949 en el conjunto de prueba.
    - En general, la implementación manual tuvo un recall ligeramente superior en la prueba, especialmente en spam (0.980 vs. 0.987 en entrenamiento y 0.933 en prueba para ambas metodologías).

    Dado que la diferencia entre ambas versiones es muy pequeña, ambas implementaciones pueden considerarse efectivas para la clasificación de mensajes spam/ham.
### **¿Por qué cree que se debe esta diferencia?**
- Las diferencias generadas en los valores pueden deberse a varios factores como el cálculo de las probabilidades y la precisión del cálculo de los logaritmos como tal.


# **TASK #3 - Clasificación de Partidas de League of Legends**

## **Referencias**

¿Qué es Support Vector Machine? (2024, octubre 4). Ibm.com. https://www.ibm.com/mx-es/topics/support-vector-machine

susmit_sekhar_bhakta Follow Improve. (2024, febrero 22). Random Forest algorithm in machine learning. GeeksforGeeks. https://www.geeksforgeeks.org/random-forest-algorithm-in-machine-learning/

What is random forest? (2024, diciembre 19). Ibm.com. https://www.ibm.com/think/topics/random-forest